In [1]:
import pandas as pd
import numpy as np
import h5py
import datetime
import math
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import seaborn as sns
from scipy.signal import savgol_filter
plt.style.use('seaborn')
%matplotlib widget
from mpl_toolkits.mplot3d import Axes3D

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from sklearn import model_selection
from sklearn import linear_model

In [2]:
def get_calibration_temperature(file, calibration_path):
    with h5py.File(file, 'r') as h5_file:
        cal_temp = h5_file[calibration_path].attrs['ftir_temperature']
        return cal_temp
    
def get_insertion_temperature(file, insertion_path):
    with h5py.File(file, 'r') as h5_file:
        ins_temp = h5_file[insertion_path].attrs['ftir_temp']
        return ins_temp
    
def get_visible_white_calibration_curve(file, calibration_path):
    with h5py.File(file, 'r') as h5_file:
        cal = h5_file[calibration_path].attrs['white_spectrum'][:]
        return cal
    
def get_ftir_white_calibration_curve(file, calibration_path):
    with h5py.File(file, 'r') as h5_file:
        cal = h5_file[calibration_path].attrs['white_spectrum2'][:]
        return cal
    
def get_visible_wavelength_vector(file, calibration_path):
    with h5py.File(file, 'r') as h5_file:
        waves = h5_file[calibration_path].attrs['spec1_wavelengths_vector'][:]
        return waves
    
def get_ftir_wavelength_vector(file, calibration_path):
    with h5py.File(file, 'r') as h5_file:
        waves = h5_file[calibration_path].attrs['spec2_wavelengths_vector'][:]
        return waves
    
def get_ftir_insertion_absorbances(file, insertion_path):
    with h5py.File(file, 'r') as h5_file:
        ins_abs = h5_file[f'{insertion_path}/spectrometer2/derived/absorbances'][:]
        return ins_abs
    
def get_ftir_insertion_raw_spectra(file, insertion_path):
    with h5py.File(file, 'r') as h5_file:
        ins_raw = h5_file[f'{insertion_path}/spectrometer2/spectra'][:]
        return ins_raw
    
def get_ftir_insertion_timestamps(file, insertion_path):
    with h5py.File(file, 'r') as h5_file:
        timestamps = h5_file[f'{insertion_path}/spectrometer2/timestamps'][:]
        return timestamps
    
def get_visible_insertion_absorbances(file, insertion_path):
    with h5py.File(file, 'r') as h5_file:
        ins_abs = h5_file[f'{insertion_path}/spectrometer1/derived/absorbances'][:]
        return ins_abs
    
def get_visible_insertion_raw_spectra(file, insertion_path):
    with h5py.File(file, 'r') as h5_file:
        ins_raw = h5_file[f'{insertion_path}/spectrometer1/spectra'][:]
        return ins_raw
    
def create_list_of_items_in_node(item_type, file, node):
    with h5py.File(file, 'r') as h5_file:
        keys = []
        if item_type == "group":
            my_type = h5py._hl.group.Group
        if item_type == "dataset":
            my_type = h5py._hl.dataset.Dataset
        h5_file[node].visit(lambda key: keys.append(key) if type(h5_file[node][key]) is my_type else None)
        return keys

def create_list_of_calibrations_in_node(file, node):
    calibrations = []
    all_groups = create_list_of_items_in_node("group", file, node)
    for group in all_groups:
        if group[-6:-3] == 'cal':
            calibrations.append(node + '/' + group)
    return calibrations

def create_list_of_insertions_in_calibration(file, calibration_path):
    with h5py.File(file, 'r') as h5_file:
        insertion_keys = list(h5_file[calibration_path].keys())
        insertions = [f'{calibration_path}/{key}' for key in insertion_keys]            
        return insertions
    
def select_by_depth_range(df, range_start, range_end):
    df_out = df.loc[(df['depth'] > range_start) & (df['depth'] < range_end)]
    return df_out

def calculate_absorbance_from_raw(raw_spectrum, white_spectrum, dark_spectrum):
    reflectance = ((raw_spectrum - dark_spectrum) / (white_spectrum - dark_spectrum))
    inverse_reflectance = 1/reflectance
    absorbance = np.log10(inverse_reflectance.astype(np.float64))
    return absorbance

def calculate_absorbance_for_2D_array(array, white_spectrum, dark_spectrum):
    absorbance_array = np.empty_like(array, dtype=np.float64)
    for i in range(array.shape[0]):
        absorbance_spectrum = calculate_absorbance_from_raw(array[i, :], white_spectrum, dark_spectrum)
        absorbance_array[i, :] = absorbance_spectrum
    return absorbance_array

def construct_full_file_path(data_path, file_name):
    file_path = data_path + file_name
    return file_path

def create_list_of_sessions_in_file(file_name):
    sessions = []
    all_groups = create_list_of_items_in_node("group", file_name, "/")
    for group in all_groups:
        if (group[0:3] == 'ses') and (len(group) == 10):
            sessions.append(group)
    return sessions

def create_list_of_insertions_in_file(file_name):
    insertions = []
    sessions = create_list_of_sessions_in_file(file_name)
    for session in sessions:
        calibrations = create_list_of_calibrations_in_node(file_name, session)
        for calibration in calibrations:
            cal_insertions = create_list_of_insertions_in_calibration(file_name, calibration)
            for insertion in cal_insertions:
                insertions.append(insertion)
    return insertions
            
def get_insertion_timestamp(file, insertion_path):
    with h5py.File(file, 'r') as h5_file:
        ins_time = h5_file[insertion_path].attrs['start_time']
        ins_timestamp = pd.Timestamp(ins_time, unit='us')
        return ins_timestamp
    
def get_calibration_timestamp(file, calibration_path):
    with h5py.File(file, 'r') as h5_file:
        cal_time = h5_file[calibration_path].attrs['calibration_start_time']
        cal_timestamp = pd.Timestamp(cal_time, unit='us')
        return cal_timestamp  
    
def find_position_in_wavelength_vector(wavelength_vector, integer):
    position = np.where(np.isclose(wavelength_vector, integer, 1e-3))[0][0]
    return position

def normalize(value, max_value, min_value):
    normalized_value = (value - min_value)/(max_value - min_value)
    return normalized_value

def get_ftir_dark_calibration_curve(file, calibration_path):
    with h5py.File(file, 'r') as h5_file:
        cal = h5_file[calibration_path].attrs['dark_spectrum2'][:]
        return cal
    
def get_visible_dark_calibration_curve(file, calibration_path):
    with h5py.File(file, 'r') as h5_file:
        cal = h5_file[calibration_path].attrs['dark_spectrum'][:]
        return cal
        
def get_ftir_spectrum_timestamp(file, insertion_path, index):
    with h5py.File(file, 'r') as h5_file:
        time = h5_file[f'{insertion_path}/spectrometer2/timestamps'][index]
        timestamp = pd.Timestamp(time, unit='us')
        return timestamp
    
def get_visible_spectrum_timestamp(file, insertion_path, index):
    with h5py.File(file, 'r') as h5_file:
        time = h5_file[f'{insertion_path}/spectrometer1/timestamps'][index]
        timestamp = pd.Timestamp(time, unit='us')
        return timestamp
    
def compute_3D_distance(x1, y1, z1, x2, y2, z2):
    distance = math.sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2)
    return distance

In [3]:
path_name = "/Users/linda/OneDrive/Documents/S4_mine_p/Projects/Data_collected/"
df = pd.read_csv('data/white_insertions.csv')
df_columns = list(df.columns.values)
df_columns.append('cal_time_0')
df_columns.append('temperature')
info_df = pd.DataFrame(columns=df_columns)
timestamps_df = pd.DataFrame(columns=['timestamp'])
columns = np.arange(0, 700, 1)
spectra = pd.DataFrame(columns=columns)
# each i represents an insertion
for i in range(df.shape[0]):
    row_file = df['file_name'][i]
    file = construct_full_file_path(path_name, row_file)
    calibration_path = df['session'][i] + "/" + df['calibration'][i]
    calibration_insertions = create_list_of_insertions_in_calibration(file, calibration_path)    
    calibration_first_timestamp = get_ftir_spectrum_timestamp(file, calibration_insertions[0], 0)
    insertion_path = calibration_path + "/" + df['insertion'][i]
    insertion_temperature = get_insertion_temperature(file, insertion_path)
     
    # raw_spectra and timestamps have many spectra and timestamps per insertion
    raw_spectra = pd.DataFrame(get_ftir_insertion_raw_spectra(file, insertion_path))    
    spectra = pd.concat([spectra, raw_spectra], axis=0, ignore_index=True)
    ts_array = get_ftir_insertion_timestamps(file, insertion_path)
    timestamps = pd.DataFrame(pd.to_datetime(ts_array, unit='us'), columns=['timestamp'])
    timestamps_df = pd.concat([timestamps_df, timestamps], axis=0, ignore_index=True)
    # info and temperature will be the same for every spectrum in insertion
    info_row = df.iloc[i:i+1, :].copy()
    info_row['cal_time_0'] = calibration_first_timestamp
    info_row['temperature'] = insertion_temperature
    # each j represents a spectrum; the info is duplicated for each spectrum
    for j in range(raw_spectra.shape[0]):
        info_df = pd.concat([info_df, info_row], axis=0, ignore_index=True)

print(info_df.shape)
print(timestamps_df.shape)
print(spectra.shape)
spectra_df = pd.concat([info_df, timestamps_df, spectra], axis=1)
print(spectra_df.shape)

(17880, 8)
(17880, 1)
(17880, 700)
(17880, 709)


In [4]:
# create the input data for the PCA.  Restricting the wavelengths to 1200 to 2200 nm
waves = get_ftir_wavelength_vector(file, calibration_path)
start_index = find_position_in_wavelength_vector(waves, 1200)
end_index = find_position_in_wavelength_vector(waves, 2200)
X = spectra_df.iloc[:, (start_index + 9):(end_index + 10)]
X.shape

(17880, 433)

In [5]:
# do the PCA
pca = PCA(n_components=10)
X_pca = pca.fit_transform(StandardScaler().fit_transform(X))

In [6]:
# The amount of variation explained per component
np.cumsum(pca.explained_variance_ratio_)

array([0.36963323, 0.60111934, 0.76741628, 0.89397149, 0.95940955,
       0.99393789, 0.99895398, 0.99944303, 0.99972842, 0.99986503])

In [7]:
fig, ax = plt.subplots()
ax.plot(np.cumsum(pca.explained_variance_ratio_[:10]));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
# Turn the components into a dataframe
X_pca_df = pd.DataFrame(X_pca)

In [9]:
# Plot of PC1 by PC2
fig, ax = plt.subplots(figsize=(10,10))
ax.set_xlim(-23, 23)
ax.set_ylim(-23, 23)
sns.scatterplot(x=X_pca_df[0], y=X_pca_df[1], hue=spectra_df['conditions'], alpha=0.2, size=2.0, palette='gist_ncar')
plt.xlabel("PC1")
plt.ylabel("PC2");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
selected = X_pca_df.loc[(spectra_df['conditions'] == 'sunshine')|(spectra_df['conditions'] == 'inside')]
selected_conditions = spectra_df.loc[(spectra_df['conditions'] == 'sunshine')|(spectra_df['conditions'] == 'inside')]


fig, ax = plt.subplots(figsize=(10,10))
ax.set_xlim(-23, 23)
ax.set_ylim(-23, 23)

sns.scatterplot(x=selected[0], y=selected[1], hue=selected_conditions['conditions'], alpha=0.2, size=selected_conditions['temperature'], palette='gist_ncar');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
selected = X_pca_df.loc[(spectra_df['conditions'] == 'heat spectrometer')|(spectra_df['conditions'] == 'cool spectrometer')]
selected_conditions = spectra_df.loc[(spectra_df['conditions'] == 'heat spectrometer')|(spectra_df['conditions'] == 'cool spectrometer')]


fig, ax = plt.subplots(figsize=(10,10))
ax.set_xlim(-23, 23)
ax.set_ylim(-23, 23)

sns.scatterplot(x=selected[0], y=selected[1], hue=selected_conditions['conditions'], alpha=0.2, size=selected_conditions['temperature'], palette='gist_ncar');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
selected1 = X_pca_df.loc[(spectra_df['conditions'] == 'heat spectrometer')]
selected2 = X_pca_df.loc[(spectra_df['conditions'] == 'cool spectrometer')]
selected3 = X_pca_df.loc[(spectra_df['conditions'] == 'heat light source')]
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection = "3d")
ax.set_xlabel("PC1")
ax.set_ylabel("PC2")
ax.set_zlabel("PC3")
ax.scatter(xs = selected1[0], ys=selected1[1], zs=selected1[2], color='tab:blue', alpha=0.2)
ax.scatter(xs = selected2[0], ys=selected2[1], zs=selected2[2], color='tab:orange', alpha=0.2)
ax.scatter(xs = selected3[0], ys=selected3[1], zs=selected3[2], color='tab:green', alpha=0.2);


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
fig, ax = plt.subplots()
sns.scatterplot(x=X_pca_df[0], y=spectra_df['temperature'], alpha=0.2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='0', ylabel='temperature'>

In [14]:
# Regress the first components against temperature
X = X_pca_df.iloc[:, 0:5]
y = spectra_df['temperature']
lr = linear_model.LinearRegression().fit(X, y)
lr.score(X, y)

0.2562013407278134

In [15]:
# computing the time delta within each insertion
timestamps1_df = spectra_df[['file_name', 'session', 'calibration', 'insertion', 'timestamp']].copy()
insertion_times = timestamps1_df.groupby(['file_name', 'session', 'calibration', 'insertion']).first()['timestamp']
spectra_delta = pd.merge(spectra_df, insertion_times, on=['file_name', 'session', 'calibration', 'insertion'])
spectra_delta['ins_time_delta'] = (spectra_delta['timestamp_x'] - spectra_delta['timestamp_y']).astype('timedelta64[ms]')/1000

In [16]:
spectra_delta.head()

,file_name,date,session,calibration,insertion,conditions,cal_time_0,temperature,timestamp_x,0,...,692,693,694,695,696,697,698,699,timestamp_y,ins_time_delta
0,210420_green_tests/green_testing_210420b.h5,4/20/21,session001,cal001,ins001,general,2021-04-20 12:29:20.206730,22.148648,2021-04-20 12:29:20.206730,0.004655,...,0.008904,0.008519,0.007887,0.007151,0.006289,0.005383,0.004043,0.002584,2021-04-20 12:29:20.206730,0.000
1,210420_green_tests/green_testing_210420b.h5,4/20/21,session001,cal001,ins001,general,2021-04-20 12:29:20.206730,22.148648,2021-04-20 12:29:22.335510,0.003627,...,0.009338,0.005970,0.005658,0.006638,0.006564,0.006130,0.004495,0.002535,2021-04-20 12:29:20.206730,2.128
2,210420_green_tests/green_testing_210420b.h5,4/20/21,session001,cal001,ins001,general,2021-04-20 12:29:20.206730,22.148648,2021-04-20 12:29:24.467705,0.003226,...,0.009734,0.010965,0.009347,0.006529,0.006881,0.008314,0.008055,0.007341,2021-04-20 12:29:20.206730,4.260
3,210420_green_tests/green_testing_210420b.h5,4/20/21,session001,cal001,ins001,general,2021-04-20 12:29:20.206730,22.148648,2021-04-20 12:29:26.584302,0.004762,...,0.011480,0.009963,0.009210,0.008778,0.007610,0.006191,0.004538,0.002822,2021-04-20 12:29:20.206730,6.377
4,210420_green_tests/green_testing_210420b.h5,4/20/21,session001,cal001,ins001,general,2021-04-20 12:29:20.206730,22.148648,2021-04-20 12:29:28.703186,0.003742,...,0.011378,0.011097,0.010056,0.008693,0.007688,0.006805,0.005380,0.003810,2021-04-20 12:29:20.206730,8.496


In [17]:
# Making a dataframe with the time delta but not the spectral data
columns = list(spectra_delta.iloc[:, :9].columns.values)
columns.append('timestamp_y')
columns.append('ins_time_delta')
spectra_info = spectra_delta[columns]

In [18]:
# Combining the information columns with the PCA output
components_info = pd.concat([spectra_info.copy(), X_pca_df], axis=1)

In [19]:
# Preparing to group the data by unique insertions (first spectrum of the insertion, then last spectrum of the insertion)
insertion_unique = ['file_name', 'session', 'calibration', 'insertion']
ins_first_components = components_info.groupby(insertion_unique).first()
ins_last_components = components_info.groupby(insertion_unique).last()

# Merging the first and the last spectrum data together
ins_first_last = pd.merge(ins_first_components, ins_last_components, on=insertion_unique)

# Reset the index to remove the hierarchical index
ins_dist = ins_first_last.reset_index()

# Calculate the 3D distance between the first and last spectrum in each insertion
# Uses the first three PC values
ins_dist['distance'] = 0.0
for i in range(ins_dist.shape[0]):
    ins_dist.iat[i, -1] = compute_3D_distance(ins_dist.at[i, '0_x'],
                                              ins_dist.at[i, '1_x'],
                                              ins_dist.at[i, '2_x'],
                                              ins_dist.at[i, '0_y'],
                                              ins_dist.at[i, '1_y'],
                                              ins_dist.at[i, '2_y'],
                                             )

In [20]:
ins_dist[['insertion', '0_x', '0_y', 'distance']]

,insertion,0_x,0_y,distance
0,ins001,14.932800,11.461957,9.231773
1,ins002,10.969851,6.494607,8.211236
2,ins003,4.904779,1.511476,6.206590
3,ins004,0.165511,-1.470325,3.864259
4,ins005,-2.936164,-3.219242,2.009117
...,...,...,...,...
293,ins076,-15.801812,-15.754259,0.143454
294,ins077,-15.841817,-15.879440,0.148726
295,ins078,-16.157105,-16.043846,0.641579
296,ins079,-16.182713,-16.181811,0.140055


In [21]:
# Calculate the time for each insertion from the first spectrum in a calibration
cal_time_delta = (ins_dist['timestamp_x_x'] - ins_dist['cal_time_0_x']).astype('timedelta64[ms]')/1000
ins_dist['cal_time_delta'] = cal_time_delta

In [22]:
fig, ax = plt.subplots()
sns.scatterplot(x = cal_time_delta, y=ins_dist['distance'], hue=ins_dist['conditions_x'], alpha=0.2);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:

g = sns.catplot(x='distance', y='conditions_x', data=ins_dist, kind='boxen', height=8, orient='h')



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
sunshine = ins_dist.loc[ins_dist['conditions_x'] == 'sunshine']

In [25]:
sunshine

,file_name,session,calibration,insertion,date_x,conditions_x,cal_time_0_x,temperature_x,timestamp_x_x,timestamp_y_x,...,2_y,3_y,4_y,5_y,6_y,7_y,8_y,9_y,distance,cal_time_delta
88,210423_green_tests/green_testing_210423.h5,session002,cal002,ins006,4/23/21,sunshine,2021-04-23 13:36:50.129068,25.593171,2021-04-23 13:36:50.129068,2021-04-23 13:36:50.129068,...,2.954840,-5.450336,-0.545896,-2.927642,3.418212,1.404763,0.268062,0.278548,8.205406,0.000
89,210423_green_tests/green_testing_210423.h5,session002,cal002,ins007,4/23/21,sunshine,2021-04-23 13:36:50.129068,26.026549,2021-04-23 13:40:20.119938,2021-04-23 13:40:20.119938,...,-3.258358,19.547087,5.009767,-3.702449,-2.798923,-0.029505,0.394799,-0.579416,15.544603,209.990
90,210423_green_tests/green_testing_210423.h5,session002,cal002,ins008,4/23/21,sunshine,2021-04-23 13:36:50.129068,26.296288,2021-04-23 13:43:03.458814,2021-04-23 13:43:03.458814,...,-2.034310,18.410650,-0.171842,-5.167762,-1.505624,0.598078,-0.235244,0.152701,11.870447,373.329
91,210423_green_tests/green_testing_210423.h5,session002,cal002,ins009,4/23/21,sunshine,2021-04-23 13:36:50.129068,26.377045,2021-04-23 13:45:47.647315,2021-04-23 13:45:47.647315,...,-2.302765,18.968228,0.471551,-4.883838,-1.738881,0.493960,-0.209502,0.069262,1.541287,537.518
92,210423_green_tests/green_testing_210423.h5,session002,cal002,ins010,4/23/21,sunshine,2021-04-23 13:36:50.129068,26.134547,2021-04-23 13:48:58.802449,2021-04-23 13:48:58.802449,...,-2.022178,18.592458,-0.076759,-5.040104,-1.544429,0.577688,-0.223713,0.124332,0.704238,728.673
93,210423_green_tests/green_testing_210423.h5,session002,cal003,ins011,4/23/21,sunshine,2021-04-23 13:52:25.643132,25.972499,2021-04-23 13:52:25.643132,2021-04-23 13:52:25.643132,...,-1.642939,18.835727,-0.022305,-5.013510,-1.578939,0.495633,-0.201070,0.147790,5.054221,0.000
94,210423_green_tests/green_testing_210423.h5,session002,cal003,ins012,4/23/21,sunshine,2021-04-23 13:52:25.643132,25.837222,2021-04-23 13:54:57.671763,2021-04-23 13:54:57.671763,...,-0.598610,15.934516,-1.963738,-5.599827,-0.597171,0.932571,-0.288508,0.448214,5.209125,152.028
95,210423_green_tests/green_testing_210423.h5,session002,cal003,ins013,4/23/21,sunshine,2021-04-23 13:52:25.643132,25.674601,2021-04-23 13:57:53.471303,2021-04-23 13:57:53.471303,...,-0.837581,16.001751,-1.775139,-5.479557,-0.797041,0.895015,-0.327261,0.393010,2.639947,327.828
96,210423_green_tests/green_testing_210423.h5,session002,cal003,ins014,4/23/21,sunshine,2021-04-23 13:52:25.643132,25.430067,2021-04-23 14:00:27.794539,2021-04-23 14:00:27.794539,...,-2.183758,20.209886,2.123049,-4.301920,-2.394529,0.147602,-0.076350,-0.252243,6.571060,482.151
97,210423_green_tests/green_testing_210423.h5,session002,cal003,ins015,4/23/21,sunshine,2021-04-23 13:52:25.643132,25.593171,2021-04-23 14:03:05.334373,2021-04-23 14:03:05.334373,...,-2.532353,20.287172,2.411828,-4.223820,-2.429393,0.098430,-0.044421,-0.281956,0.990766,639.691
